In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import os
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [ ]:
from app.utils.data_loader import DataLoader
from app.utils.health_score import HealthScore
from app.utils.portfolios import Portfolios
from app.utils.portfolios_repo import PortfoliosRepository
from app.utils.portfolios_service import PortfolioService
from app.utils.portprop_matrices import PortpropMatrices
from app.utils.portprop_matrices_repo import PortpropMatricesRepository
from app.utils.rebalancer import Rebalancer
from app.utils.rebalancer_repo import RebalancerRepository

## Load Data

In [16]:
load_dotenv()  # Load environment variables from .env file
print(os.getenv('LOAD_DATA_FROM_DWH'))

False


In [21]:
data_loader=DataLoader(load_from_db=True)
ppm_repo = PortpropMatricesRepository(data_loader=data_loader)
ports_repo = PortfoliosRepository(data_loader=data_loader)
rebalancer_repo = RebalancerRepository(data_loader=data_loader)

In [22]:
client_out_enriched = ports_repo.load_client_out_product_enriched(
    as_of_date="2025-08-31"
)
client_styles = ports_repo.load_client_style(
    as_of_date="2025-08-31"
)

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.


In [23]:
ports_ref_table = {
            'product_mapping' : ports_repo.load_product_mapping(),
            'product_underlying' : ports_repo.load_product_underlying(),
}

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.


In [24]:
ppm_ref_dict = {
    "portprop_factsheet": ppm_repo.load_portprop_factsheet(),
    "portprop_benchmark": ppm_repo.load_portprop_benchmark(),
    "portprop_ge_mapping": ppm_repo.load_portprop_ge_mapping(),
    "portprop_fallback": ppm_repo.load_portprop_fallback(),
    "portprop_ret_eow": ppm_repo.load_portprop_ret_eow(),
    "advisory_health_score": ppm_repo.load_advisory_health_score(),
}

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.


In [25]:
rb_ref_dict = {
    "es_sell_list": rebalancer_repo.load_es_sell_list(),
    "product_recommendation_rank_raw": rebalancer_repo.load_product_recommendation_rank_raw(),
    "mandate_allocation": rebalancer_repo.load_mandate_candidates(),
}

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.


## Instances

In [26]:
## Portsfolios
ports_all = Portfolios()
ports_all.set_ref_tables(ports_ref_table)
df_out, df_style, port_ids, port_id_mapping = ports_all.create_portfolio_id(client_out_enriched, client_styles, column_mapping=['as_of_date', 'customer_id'])
ports_all.set_portfolio(df_out, df_style, port_ids, port_id_mapping)

## Portfolio Service
port_service = PortfolioService(ports_all)

## Portprop Matrices
ppm = PortpropMatrices(ppm_ref_dict)

## Health Score
hs = HealthScore()

## Rebalancer
rb = Rebalancer()
rb.set_ref_tables(rb_ref_dict)


## Portfolio (Service)

In [27]:
## get list of all customer ids
print(port_service.get_all_customer_ids())

[45529, 26326, 23349, 24149, 28359, 31843, 13083, 18051, 43495, 12346, 27766, 23905, 28999, 36170, 33624, 23356, 27144, 30084, 14154, 26352, 35059, 11838, 30030, 45708, 36996, 24032, 36477, 31197, 30611, 28587, 30961, 31554, 32215, 32522, 29558, 33430, 33677, 33901, 34365, 35258, 36830, 36998, 15211, 38028, 31762, 16902, 36813, 43246, 43929, 46604, 47420, 47470, 44605, 48844, 45701, 50044, 50745, 27547, 51637, 51609, 52747, 53638, 54541, 56592, 56897, 58055, 58312, 59883, 60765, 59926, 61749, 60538, 67451, 72106, 72434, 70458, 73696, 74374, 54552, 76048, 76316, 76011, 54431, 73480, 72660, 77281, 76457, 77074, 79731, 80089, 71821, 33739, 81551, 81757, 81853, 81977, 77198, 80100, 86279, 88937, 90104, 78452, 90017, 89677, 94184, 95105, 95067, 94903, 57414, 96146, 94315, 96504, 97699, 96398, 99466, 100020, 53015, 35261, 102245, 102085, 101993, 96931, 103542, 102804, 104572, 104615, 105205, 60577, 107010, 106818, 67412, 107957, 108382, 47880, 109029, 110138, 77057, 17739, 97845, 347, 111187

In [28]:
## get single port from customer id
port = port_service.get_client_portfolio(customer_id=12560)

In [30]:
## get port outstanding
port.df_out

,customer_id,as_of_date,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,...,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,value,port_id,asset_class_code,weight
1981,12560,2025-08-31,B00174068,NTL26NA,TRADE,L,THB,NTL26NA,Fixed Income,Fixed Income,...,True,0.0250,True,<NA>,Not Top-Pick,<NA>,2.065495e+06,1147,AA_FI,0.012518
1985,12560,2025-08-31,B00174420,CPF23PA,TRADE,L,THB,CPF23PA,Fixed Income,Fixed Income,...,True,0.0250,True,<NA>,Not Top-Pick,<NA>,2.138060e+06,1147,AA_FI,0.012958
1986,12560,2025-08-31,B00200786,GULF284A,TRADE,L,THB,GULF284A,Fixed Income,Fixed Income,...,True,0.0250,True,<NA>,Not Top-Pick,<NA>,2.113839e+06,1147,AA_FI,0.012811
1990,12560,2025-08-31,C00020406,GIS USD Collateral,GIS,L,USD,GIS USD Collateral,Cash,Cash and Cash Equivalent,...,True,0.0480,True,<NA>,Not Top-Pick,<NA>,1.895890e+06,1147,AA_CASH,0.011490
2006,12560,2025-08-31,C00046338,SA_2004738659,TRADE,L,THB,KKP Savvy,Cash,Cash and Cash Equivalent,...,True,0.0141,True,<NA>,Not Top-Pick,<NA>,0.000000e+00,1147,AA_CASH,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75579,12560,2025-08-31,M00316416,USDAILY,TRADE,L,USD,USDAILY,Mutual Fund,Fixed Income,...,False,0.0480,True,<NA>,Not Top-Pick,<NA>,3.239966e+03,1147,AA_FI,0.000020
75589,12560,2025-08-31,P00156097,ODL,GIS,L,USD,ODL,Private Market,Global Equity,...,True,0.0770,True,<NA>,Not Top-Pick,<NA>,7.256333e+06,1147,AA_GE,0.043977
75726,12560,2025-08-31,S00237377,VAYU1,TRADE,L,THB,VAYU1,Listed Securities,Local Equity,...,False,0.0420,True,<NA>,Not Top-Pick,<NA>,1.590000e+07,1147,AA_LE,0.096362
75731,12560,2025-08-31,S00270995,AOR:arcx,SAXO,L,USD,iShares Core 60/40 Balanced Allocation ETF,Listed Securities,Global Equity,...,True,0.0770,True,<NA>,Top-Pick,<NA>,5.937606e+05,1147,AA_GE,0.003599


In [31]:
# get port allocation lookthrough
port.get_portfolio_asset_allocation_lookthrough(ppm)

asset_class,port_id,aa_alt,aa_cash,aa_fi,aa_ge,aa_le
0,1147,0.060932,0.041909,0.400828,0.390025,0.106306


In [32]:
# get model allocation 
port.get_model_asset_allocation_lookthrough(ppm)

,port_id,port_investment_style,portpop_styles,aa_alt_model,aa_cash_model,aa_fi_model,aa_ge_model,aa_le_model
0,1147,Moderate High Risk,Medium to Moderate High Risk,0.13,0.06,0.27,0.45,0.05


## Healthscore

In [33]:
## get client health score
health_score, health_score_comp = port.get_portfolio_health_score(ppm,hs)

In [34]:
health_score['health_score'].values[0]

np.float64(8.0)

## Rebalancer

In [35]:
rb.rebalance(port, ppm, hs)

,transaction_no,batch_no,port_id,product_id,src_sharecodes,desk,port_type,currency,value,weight,flag,expected_weight,action,amount
0,1,1,1147,S00087551,BTSGIF,TRADE,L,THB,3545000.0,0.021485,not_monitored_product,0.000000,sell,-3.545000e+06
1,2,1,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,NaN,NaN,cash_proxy_funding,NaN,funding,3.545000e+06
2,3,2,1147,S00237377,VAYU1,TRADE,L,THB,15900000.0,0.096362,not_monitored_product,0.000000,sell,-1.590000e+07
3,4,2,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,NaN,NaN,cash_proxy_funding,NaN,funding,1.590000e+07
4,5,3,1147,DTHB00000000,KKPBAPLUSTHB,TRADE,L,THB,0.0,0.000000,discretionary_buy,0.090553,buy,1.494139e+07
5,6,3,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,NaN,NaN,cash_proxy_funding,NaN,funding,-1.494139e+07


In [36]:
port.df_out

,port_id,product_id,src_sharecodes,desk,port_type,currency,value,product_display_name,product_type_desc,asset_class_name,...,is_risky_asset,coverage_prdtype,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,asset_class_code,weight
0,1147,B00138028,MINT23PA,TRADE,L,THB,4.133600e+06,MINT23PA,Fixed Income,Fixed Income,...,True,N/A,True,0.025,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.025052
1,1147,B00138680,TUC282A,TRADE,L,THB,2.026721e+06,TUC282A,Fixed Income,Fixed Income,...,True,N/A,True,0.025,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.012283
2,1147,B00143744,BGRIM23PA,TRADE,L,THB,2.099210e+06,BGRIM23PA,Fixed Income,Fixed Income,...,True,N/A,True,0.025,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.012722
3,1147,B00145295,LOTUSS284A,TRADE,L,THB,3.194455e+06,LOTUSS284A,Fixed Income,Fixed Income,...,True,N/A,True,0.025,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.019360
4,1147,B00151378,SIRI25DA,TRADE,L,THB,2.004395e+06,SIRI25DA,Fixed Income,Fixed Income,...,True,N/A,True,0.040,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.012148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,1147,T00281178,XS3004384742,GIS,L,USD,9.804772e+05,SHFIN20250221A,Structured Note,Global Equity,...,True,N/A,True,0.077,True,<NA>,Not Top-Pick,<NA>,AA_GE,0.005942
71,1147,T00305836,XS2993068878,GIS,L,USD,2.617949e+06,SHFIN20250401A,Structured Note,Global Equity,...,True,N/A,True,0.077,True,<NA>,Not Top-Pick,<NA>,AA_GE,0.015866
72,1147,T00310048,XS3048279049,GIS,L,USD,1.927777e+06,SHFIN20250428A,Structured Note,Global Equity,...,True,N/A,True,0.077,True,<NA>,Not Top-Pick,<NA>,AA_GE,0.011683
73,1147,T00320952,XS3102117291,GIS,L,USD,1.599890e+06,SHFIN20250630A,Structured Note,Global Equity,...,True,N/A,True,0.077,True,<NA>,Not Top-Pick,<NA>,AA_GE,0.009696
